In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install fastBPE
!pip install fairseq
!pip install unidecode
!pip install underthesea
!pip install phased_lstm_keras 



In [ ]:
import re
import os
import numpy as np
import torch
import pickle
import itertools
import underthesea
from tqdm import tqdm
from nltk import ngrams
import tensorflow as tf
from unidecode import unidecode
from tensorflow.keras import layers
from keras.models import Sequential
from fairseq.data import Dictionary
from tensorflow.keras.optimizers import Adam
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from sklearn.model_selection import train_test_split
from transformers import RobertaConfig,TFRobertaModel






In [ ]:
# Download Pretrained PhoBert
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz


--2022-01-16 04:16:14--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.226.52.126, 13.226.52.20, 13.226.52.43, ...
Connecting to public.vinai.io (public.vinai.io)|13.226.52.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  75.0MB/s    in 15s     

2022-01-16 04:16:32 (77.6 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [ ]:
# Load dữ liệu từ file Data.pkl
data = pickle.load(open('/content/gdrive/MyDrive/Spell Check PhoBert/Data.pkl','rb'))
alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'
#data = data.replace("\n",".")
training_data = []
sentences = data.split("\n")
for j in tqdm(sentences):
      if len(j.split()) > 5 and re.match(alphabet, j.lower()):
          training_data.append(j)
print(training_data[0])

100%|██████████| 377320/377320 [00:10<00:00, 34790.90it/s]

Do vậy, mở rộng cửa sổ tạo luồng không khí lưu thông, nhiệt độ phòng cao, có ánh nắng mặt trời sẽ giúp giảm đời sống virus., 


In [ ]:
# Lựa chọn các đoạn văn bản lớn năm từ
def extract_phrases(text):
    return re.findall(r'\w[\w ]+', text)
phrases = itertools.chain.from_iterable(extract_phrases(text) for text in training_data)
phrases = [p.strip() for p in phrases if len(p.split()) > 5]
def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)
_save_pkl('phrases.pkl', phrases)
print(phrases[0:10])


['mở rộng cửa sổ tạo luồng không khí lưu thông', 'có ánh nắng mặt trời sẽ giúp giảm đời sống virus', 'Các chuyên gia khuyến cáo mỗi người nên tự ý thức giữ vệ sinh cá nhân', 'bên cạnh việc đeo khẩu trang', 'rửa tay thường xuyên bằng dung dịch sát khuẩn như xà phòng', 'có thể ngăn 44 các bệnh truyền nhiễm', 'Sau khi tiếp xúc với bề mặt công cộng', 'nên rửa tay và vệ sinh cá nhân kỹ để hạn chế việc mang vi khuẩn', 'bụi bẩn từ môi trường ngoài vào nhà ở', 'Với những đồ dùng cá nhân như điện thoại']


In [ ]:
# Hàm tạo lỗi cho các câu
alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'
letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")
telex={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}
vni={"ă":"a8","â":"a6","á":"a1","à":"a2","ả":"a3","ã":"a4","ạ":"a5","ắ":"a81","ổ":"o63","ỗ":"o64","ộ":"o65","ơ":"o7",
"ằ":"a82","ẳ":"a83","ẵ":"a84","ặ":"a85","ó":"o1","ò":"o2","ỏ":"o3","õ":"o4","ọ":"o5","ô":"o6","ố":"o61","ồ":"o62",
"ớ":"o71","ờ":"o72","ở":"o73","ỡ":"o74","ợ":"o75","é":"e1","è":"e2","ẻ":"e3","ẽ":"e4","ẹ":"e5","ê":"e6","ế":"e61","ề":"e62",
"ể":"e63","ễ":"e64","ệ":"e65","ú":"u1","ù":"u2","ủ":"u3","ũ":"u4","ụ":"u5","ư":"u7","ứ":"u71","ừ":"u72","ử":"u73","ữ":"u74",
"ự":"u75","í":"i1","ì":"i2","ỉ":"i3","ị":"i5","ĩ":"i4","ý":"y1","ỳ":"y2","ỷ":"y3","ỵ":"y5","đ":"d6",
"Ă":"A8","Â":"A6","Á":"A1","À":"A2","Ả":"A3","Ã":"A4","Ạ":"A5","Ắ":"A81","Ổ":"O63","Ỗ":"O64","Ộ":"O65","Ơ":"O7",
"Ằ":"A82","Ẳ":"A83","Ẵ":"A84","Ặ":"A85","Ó":"O1","Ò":"O2","Ỏ":"O3","Õ":"O4","Ọ":"O5","Ô":"O6","Ố":"O61","Ồ":"O62",
"Ớ":"O71","Ờ":"O72","Ở":"O73","Ỡ":"O74","Ợ":"O75","É":"E1","È":"E2","Ẻ":"E3","Ẽ":"E4","Ẹ":"E5","Ê":"E6","Ế":"E61","Ề":"E62",
"Ể":"E63","Ễ":"E64","Ệ":"E65","Ú":"U1","Ù":"U2","Ủ":"U3","Ũ":"U4","Ụ":"U5","Ư":"U7","Ứ":"U71","Ừ":"U72","Ử":"U73","Ữ":"U74",
"Ự":"U75","Í":"I1","Ì":"I2","Ỉ":"I3","Ị":"I5","Ĩ":"I4","Ý":"Y1","Ỳ":"Y2","Ỷ":"Y3","Ỵ":"Y5","Đ":"D6"}
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}
vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")


def add_noise(sentence, pivot1,pivot2):
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        else:  
            random = np.random.uniform(0,1,1)[0]   
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in telex.keys() and sentence[i] in vni.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.5:
                        noisy_sentence+=telex[sentence[i]]
                    elif random2<0.7:
                        noisy_sentence += vni[sentence[i]]
                    elif random2<0.9:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in telex.keys():#Lỗi Telex
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.5:
                        noisy_sentence+=telex[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in vni.keys():#Lỗi VNI
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=vni[sentence[i]]
                    elif random4<0.95 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys(): #Sai dấu vùng miền
                    random5=np.random.uniform(0,1,1)[0]
                    if random5<=0.5:
                        noisy_sentence+=region[sentence[i]]
                    elif random5<0.9:
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random6=np.random.uniform(0,1,1)[0]
                        if random6<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass
      
        i += 1
    return noisy_sentence


In [ ]:
def gen_ngrams(words, n=5):
    return ngrams(words.split(), n)
list_ngrams = []
for p in tqdm(phrases):
    if not re.match(alphabet, p.lower()):
        continue
    for ngr in gen_ngrams(p, 5):
        if len(" ".join(ngr)) < 30:
            list_ngrams.append(" ".join(ngr))
            

100%|██████████| 706184/706184 [00:13<00:00, 50756.87it/s]


In [ ]:

del phrases
del training_data
list_ngrams = list((list_ngrams))
print(list_ngrams[0:10])
print(len(list_ngrams))

['mở rộng cửa sổ tạo', 'rộng cửa sổ tạo luồng', 'cửa sổ tạo luồng không', 'sổ tạo luồng không khí', 'tạo luồng không khí lưu', 'luồng không khí lưu thông', 'có ánh nắng mặt trời', 'ánh nắng mặt trời sẽ', 'nắng mặt trời sẽ giúp', 'mặt trời sẽ giúp giảm']
6316221


In [ ]:
print(add_noise('thành phố như hiện nay',0.94,0.985))

thành phố như hiện nay


In [ ]:
# Load model PhoBert
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT.eval()
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT


In [ ]:
data_train, data_val = train_test_split(list_ngrams[:10000],test_size=0.15, random_state=42)
#Tạo ra một cặp câu lỗi và câu bình thường
train_data=[]
for i in tqdm(range(len(data_train))):
    double = []
    double.insert(0,add_noise(data_train[i],0.9,0.95))
    double.append(data_train[i])
    train_data.append(double)
eval_data=[]
for i in tqdm(range(len(data_val)):
    double = []
    double.insert(0,add_noise(data_val[i],0.9,0.95))
    double.append(data_val[i])
    eval_data.append(double)

100%|██████████| 8500/8500 [00:01<00:00, 7193.17it/s]


In [ ]:
# Encode sent in list-gram
max_sequence_length = 40 
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in enumerate(lines): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
    
    outputs[idx,:] = np.array(input_ids)
  return outputs
vocab = Dictionary()
vocab.add_from_file("PhoBERT_base_fairseq/dict.txt")
train_dataset = []
print(train_data[1])
for i in tqdm(range(len(train_data))):
  [x1,x2] = convert_lines(train_data[i],vocab,phoBERT.bpe)
  train_dataset.append([x1,x2])
# print(train_dataset[1])
# print('x1 tensor encode: {}, shape: {}'.format(train_dataset[1][0][:10], train_dataset[1][0].size))
# print('x1 tensor decode: ', phoBERT.decode(torch.tensor(train_dataset[1][0]))[:103])
# print('x2 tensor encode: {}, shape: {}'.format(train_dataset[1][1][:10], train_dataset[1][1].size))
# print('x2 tensor decode: ', phoBERT.decode(torch.tensor(train_dataset[1][1]))[:103])
# lines = ['Học_sinh được nghỉ học bắt dầu từ tháng 3 để tránh dịch covid-19', 'số lượng ca nhiễm bệnh đã giảm bắt đầu từ tháng 5 nhờ biện pháp mạnh tay']
# [x1, x2] = convert_lines(lines, vocab, phoBERT.bpe)


['rộng cửa sổ tạo luog', 'rộng cửa sổ tạo luồng']


100%|██████████| 8500/8500 [00:01<00:00, 4725.42it/s]


In [ ]:
# Tạo DataLoader để quản lý dữ liệu đưa vào huấn luyện
BATCH_SIZE=128
for x,y in train_dataset:
  dataset = torch.utils.data.TensorDataset(torch.tensor(x,dtype=torch.long), torch.tensor(y,dtype=torch.long))
  dataset = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model = BertForMaskedLM.from_pretrained('vinai/phobert-base')

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing BertForMaskedLM: ['roberta.encoder.layer.11.attention.self.query.weight', 'roberta.encoder.layer.3.output.LayerNorm.bias', 'roberta.encoder.layer.4.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.encoder.layer.11.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.dense.bias', 'roberta.encoder.layer.0.intermediate.dense.bias', 'roberta.encoder.layer.2.output.dense.weight', 'roberta.encoder.layer.3.intermediate.dense.weight', 'roberta.encoder.layer.10.output.dense.bias', 'roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.9.intermediate.dense.weight', 'roberta.encoder.layer.11.output.LayerNorm.weight', 'roberta.encoder.layer.10.output.dense.weight', 'roberta.encoder.layer.7.attention.self.key.weight', 'roberta.encoder.layer.7.output.LayerNorm.bias', 'roberta.encoder.layer.1.intermediate.dense.weight', 'rober

In [ ]:
# input_ids = []
# attention_masks = []

# # # For every sentence...
# # for sent in sentences:
# #     # `encode_plus` will:
# #     #   (1) Tokenize the sentence.
# #     #   (2) Prepend the `[CLS]` token to the start.
# #     #   (3) Append the `[SEP]` token to the end.
# #     #   (4) Map tokens to their IDs.
# #     #   (5) Pad or truncate the sentence to `max_length`
# #     #   (6) Create attention masks for [PAD] tokens.
# #     encoded_dict = tokenizer.encode_plus(
# #                         sent,                      # Sentence to encode.
# #                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'
# #                         max_length = 64,           # Pad & truncate all sentences.
# #                         pad_to_max_length = True,
# #                         return_attention_mask = True,   # Construct attn. masks.
# #                         return_tensors = 'pt',     # Return pytorch tensors.
# #                    )
    
# #     # Add the encoded sentence to the list.    
# #     input_ids.append(encoded_dict['input_ids'])
    
# #     # And its attention mask (simply differentiates padding from non-padding).
# #     attention_masks.append(encoded_dict['attention_mask'])
# encoded_dict = tokenizer.encode_plus(train_data[0][0],                      # Sentence to encode.
#                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                          max_length = 40,           # Pad & truncate all sentences.
#                          pad_to_max_length = True,
#                          return_attention_mask = True, )  # Construct attn. masks.)

# # Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)

In [ ]:
del train_df,eval_df

In [ ]:
# Huấn luyện thử bằng thư viện SimpleTransformers với model Seq2Seq
import logging
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.DataFrame(train_data, columns=["input_text", "target_text"])

eval_df = pd.DataFrame(eval_data, columns=["input_text", "target_text"])



100%|██████████| 8500/8500 [00:01<00:00, 6758.29it/s]

                    input_text              target_text
0            ở rộng cửa sổ tạo       mở rộng cửa sổ tạo
1        rộng cửa sổ gạo luồng    rộng cửa sổ tạo luồng
2       cửa sổ tajo luồn không   cửa sổ tạo luồng không
3       sổ tạo luồng không khí   sổ tạo luồng không khí
4      tạo luồng kôhng khí lưu  tạo luồng không khí lưu
...                        ...                      ...
8495  thơm nhiều hosa thực vật  thơm nhiều hóa thực vật
8496     nhiều hóa thực vật và    nhiều hóa thực vật và
8497      hóa thực vật và tinh     hóa thực vật và tinh
8498      thực vật và tinh dầu     thực vật và tinh dầu
8499       vật và tinh dầu bpc      vật và tinh dầu bạc

[8500 rows x 2 columns]


In [ ]:
# Huấn luyện thử bằng thư viện SimpleTransformers với model Seq2Seq
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 10,
    "train_batch_size": 128,
    "num_train_epochs": 2,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 40,
    "manual_seed": 4,
}

encoder_type = "roberta"

model = Seq2SeqModel(
    encoder_type,
    "roberta-base",
    "vinai/phobert-base",
    args=model_args,
    use_cuda=False,
)

model.train_model(train_df)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['roberta.encoder.layer.6.crossattention.self.query.weight', 'roberta.encoder.layer.2.crossattention.self.key.bias', 'roberta.encoder.layer.11.crossattention.output.d

  0%|          | 0/8500 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_utils: Saving features into cached file cache_dir/roberta-base-vinai_phobert-base_cached_108500
INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/67 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:524: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Running Epoch 1 of 2:   0%|          | 0/67 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of roberta-base-vinai/phobert-base model complete. Saved to outputs/.


(134, 1.1447084008259074)